## Topographic Complexity/Variability: Roughness  
Developed in November 2023 by Dr. Larry Syu-Heng Lai (University of Washington)  

Recommended reference:
* Wilson, M.F.J., O’Connell, B., Brown, C., Guinan, J.C., Grehan, A.J., 2007. Multiscale Terrain Analysis of Multibeam Bathymetry Data for Habitat Mapping on the Continental Slope. Marine Geodesy 30, 3-35. https://doi.org/10.1080/01490410701295962 

### Initial setup

In [ ]:
import numpy as np
import rasterio
from tqdm.notebook import tqdm
from scipy.ndimage import maximum_filter, minimum_filter

### Define data path

In [ ]:
# Define your file paths and file names separately
input_folder = '/Users/larryslai/Library/CloudStorage/Dropbox/QGIS/WA LiDAR/'
#input_file_name = 'Test_DEM.tif'
input_file_name = 'Tokeland_DEM.tif'
#input_file_name = 'Nemah_DEM.tif'
#input_file_name = 'Francies_DEM.tif'

output_folder = '/Users/larryslai/Library/CloudStorage/Dropbox/QGIS/WA LiDAR/'
#output_file_name = 'Test_pyRoughness.tif'
output_file_name = 'Tokeland_pyRoughness.tif'
#output_file_name = 'Nemah_pyRoughness.tif'
#output_file_name = 'Francies_pyRoughness.tif'

# Combine folder and file names to create the full paths
input_tif_path = input_folder + input_file_name
output_tif_path = output_folder + output_file_name

### Read a DEM

In [ ]:
with rasterio.open(input_tif_path) as src:
    dem = src.read(1)  # Read the first band into a 2D array
    meta = src.meta

See coordinate system info of the GeoTIFF

In [ ]:
# Open the GeoTIFF file
with rasterio.open(input_tif_path) as src:
    # Read the CRS
    crs = src.crs
    
    # Print the CRS information
    print(f"CRS: {crs}")
    print(f"CRS as WKT: {crs.wkt}")
    print(f"CRS as PROJ string: {crs.to_proj4()}")
    print(f"CRS as EPSG code: {crs.to_epsg()}")
    print(f"CRS as dictionary: {crs.to_dict()}")

## Roughness (R)

Roughness (R) is calculated as the difference between the maximum and minimum elevation values within a specified window size surrounding each pixel:

$$
R(n) = B_{\text{max}} - B_{\text{min}}
$$

where:
- $ B_{\text{max}}(n) $ is the maximum elevation in the $ n \times n $ window.
- $ B_{\text{min}}(n) $ is the minimum elevation in the $ n \times n $ window.


##### Rugosity function  

In [ ]:
def calculate_roughness(dem, window_size):
    """
    Calculate the roughness for each cell in the raster.
    
    :param dem: 2D array of elevation values.
    :param window_size: Size of the moving window to calculate the roughness.
    :return: 2D array of roughness values.
    """
    # Calculate maximum and minimum within the moving window
    max_elevation = maximum_filter(dem, size=window_size, mode='nearest')
    min_elevation = minimum_filter(dem, size=window_size, mode='nearest')
    
    # Roughness is the difference between maximum and minimum elevation values
    roughness = max_elevation - min_elevation
    
    return roughness

Calculate Roughness with a given window size

In [ ]:
# Calculate with a N x N window as an example
window_size = 3  # Replace with the desired window size

# Initialize a progress bar
with tqdm(total=dem.size, desc='Calculating Roughness') as pbar:
    # Calculate roughness for the entire DEM
    roughness_map = calculate_roughness(dem, window_size)
    pbar.update(dem.size)


Output data into GeoTIFF
* Enabling geotiff compression to reduce writing time
* Enabling Tile-based writing if needed
* Enabling BIGTIFF parameter to allow writing a large GeoTIFF

In [ ]:
# Update metadata for output GeoTIFF
meta.update(dtype=rasterio.float32, compress='lzw', tiled=True, bigtiff='IF_SAFER')

# Write Roughness to a new GeoTIFF
with rasterio.open(output_tif_path, 'w', **meta) as dst:
    dst.write(roughness_map.astype(rasterio.float32), 1)